In [1]:
import time, numpy, pandas, datetime

In [5]:
df = pandas.read_csv('./features.csv', header = 0, index_col='match_id')
y = df[['radiant_win']].values
df = df.ix[:,:-6]
df.head()

In [ ]:
count_na = df.isnull().sum()
count_na[count_na>0]

In [ ]:
df = df.fillna(0)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=117)
scores = []
numbers = [5, 10, 20, 30, 40, 200, 300]
for n in numbers:
    start_time = datetime.datetime.now()
    model = GradientBoostingClassifier(n_estimators=n, random_state=117)
    model_roc = cross_val_score(model, df, y, cv=cv, scoring='roc_auc')
    print n, 'trees. Time elapsed:', datetime.datetime.now() - start_time
    print model_roc
    scores.append( (n, numpy.mean(model_roc))  )
print scores

In [ ]:
del scores
del model_roc

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
scaler = StandardScaler()
df_s = scaler.fit_transform(df)

cv = KFold(n_splits=5, shuffle=True, random_state=117)

powers = range(-5, 6)
coefs = [10.0 ** i for i in powers]
scores = []
for C in coefs:
    start_time = datetime.datetime.now()
    model = LogisticRegression(C=C, random_state=117)
    model_roc = cross_val_score(model, df_s, y, cv=cv, scoring='roc_auc')
    print 'C = ', C, '. Time elapsed:', datetime.datetime.now() - start_time
    print model_roc
    scores.append( (C, numpy.mean(model_roc)) )
print scores

In [ ]:
del df_s
del scores
del model_roc

In [ ]:
df_c = df.copy()
del df_c['lobby_type']
for n in xrange(1, 6):
        del df_c['r{}_hero'.format(n)]
        del df_c['d{}_hero'.format(n)]

In [ ]:
scaler = StandardScaler()
arr_c = scaler.fit_transform(df_c)

cv = KFold(n_splits=5, shuffle=True, random_state=117)

powers = range(-5, 6)
coefs = [10.0 ** i for i in powers]
scores2 = []
for C in coefs:
    start_time = datetime.datetime.now()
    model = LogisticRegression(C=C, random_state=117)
    model_roc = cross_val_score(model, arr_c, y, cv=cv, scoring='roc_auc')
    print 'C = ', C, '. Time elapsed:', datetime.datetime.now() - start_time
    print model_roc
    scores2.append( (C, numpy.mean(model_roc)) )
print scores2

In [ ]:
heroes = df[['r1_hero', 'r2_hero', 'r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']]
N = max(numpy.unique(heroes))
print N

In [ ]:
X_pick = numpy.zeros((df.shape[0], N))
for i, match_id in enumerate(df.index):
    for p in xrange(5):
        X_pick[i, df.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
df_binary = pandas.DataFrame(X_pick, index=df.index)
df_binary.head()

In [ ]:
df_tot = pandas.merge(df_c, df_binary, left_index = True, right_index = True)
del df
del df_c
del df_binary

In [ ]:
scaler = StandardScaler()
arr_tot = scaler.fit_transform(df_tot)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=117)

powers = range(-5, 6)
coefs = [10.0 ** i for i in powers]
scores3 = []
for C in coefs:
    start_time = datetime.datetime.now()
    model = LogisticRegression(C=C, random_state=117)
    model_roc = cross_val_score(model, arr_tot, y, cv=cv, scoring='roc_auc')
    print 'C = ', C,'. Time elapsed:', datetime.datetime.now() - start_time
    print model_roc
    scores3.append( (C, numpy.mean(model_roc)) )
print scores3

In [ ]:
del scores3
del model_roc

In [ ]:
df_test = pandas.read_csv('./features_test.csv', header = 0, index_col='match_id')
df_test = df_test.fillna(0)

X_pick = numpy.zeros((df_test.shape[0], 112))
for i, match_id in enumerate(df_test.index):
    for p in xrange(5):
        X_pick[i, df_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
df_test_binary = pandas.DataFrame(X_pick, index=df_test.index)

del df_test['lobby_type']
for n in xrange(1, 6):
        del df_test['r{}_hero'.format(n)]
        del df_test['d{}_hero'.format(n)]
df_test_tot = pandas.merge(df_test, df_test_binary, left_index = True, right_index = True)

In [ ]:
scaler = StandardScaler()
arr_tot = scaler.fit_transform(df_tot)
arr_test_tot = scaler.transform(df_test_tot)
model = LogisticRegression(C=0.01, random_state=117)
model.fit(arr_tot, y)
del arr_tot
a = model.predict_proba(arr_test_tot)[:,1]

In [ ]:
print a.max(), a.min()